In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import os.path

In [2]:
# Definisci la tua user agent
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

# Definisci gli headers
HEADERS = {'User-Agent': USER_AGENT, 'Accept-Language': 'en-US, en;q=0.5'}

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
}

In [3]:
# Function which defines o header for the connection with amazon.com and makes a request to extract the data
def make_request(num_page):
    
    # Header to avoid the problem of non-secure connection with amazon.com
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
               "Accept-Encoding":"gzip, deflate", 
               "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
               "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"} 

    # Request the desired number of pages
    request = requests.get('https://www.amazon.it/s?k=laptops&page='+str(num_page), headers = headers)
    
    # Then extract the content
    content = request.content
    
    return content

In [4]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = None

    return title_string


In [5]:
# Function to extract Product Price
def get_price(soup):
    try:
        # Locate the price container
        price_container = soup.find("div", attrs={'id': 'corePriceDisplay_desktop_feature_div'})

        # Check if it has a standard price
        price = price_container.find("span", class_='a-price-whole').text.strip()
        price_fraction = price_container.find("span", class_='a-price-fraction').text.strip()
        price = price + price_fraction

    except AttributeError:
        price = None

    return price

In [6]:
def get_not_discounted_price(soup):
    try:
        full_price_tags = soup.find_all("span", class_="a-size-small aok-offscreen")
        if len(full_price_tags) >= 2:
            return full_price_tags[1].text.strip()
        else:
            return None
    except AttributeError:
        return None

In [7]:
# Function to extract Discount
def get_discount(soup):
    try:
            discount_tag = soup.find("span", class_="a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage")
            if discount_tag:
                return discount_tag.text.strip()
            else:
                return None
    except AttributeError:
        return None

In [8]:
# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = None

    return rating

In [9]:
# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [10]:
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = None

    return review_count

In [11]:
def get_technical_details(soup):
    details = {
        "Marca": None,
        "Produttore": None,
        "serie": None,
        "colore": None,
        "Dimensioni del collo": None,
        "Pile": None,
        "Numero modello articolo": None,
        "Fattore di forma": None,
        "Dimensione schermo": None,
        "Risoluzione schermo": None,
        "Risoluzione": None,
        "Marchio processore": None,
        "Tipo processore": None,
        "Velocità processore": None,
        "Numero processori": None,
        "Dimensione RAM": None,
        "Tecnologia di memoria": None,
        "Tipologia di memoria computer": None,
        "Memoria massima supportata": None,
        "Dimensioni Hard-Disk": None,
        "Descrizione Hard-Disk": None,
        "Interfaccia Hard-Disk": None,
        "Dettagli audio": None,
        "Coprocessore grafico": None,
        "Grafica Chipset Brand": None,
        "Descrizione scheda grafica": None,
        "Tipo memoria scheda grafica": None,
        "Dimensioni memoria scheda grafica": None,
        "Interfaccia scheda grafica": None,
        "Tipo wireless": None,
        "Numero di porte USB 2.0": None,
        "Numero di porte USB 3.0": None,
        "Numero di porte HDMI": None,
        "Numero di porte Ethernet": None,
        "Voltaggio": None,
        "Wattaggio": None,
        "Tipologia di periferica ottica": None,
        "Posteriore Webcam Risoluzione": None,
        "Frontale Webcam Risoluzione": None,
        "Sistema operativo": None,
        "Le batterie sono incluse": None,
        "Capacità della batteria al litio": None,
        "La vita media della batteria (in ore)": None,
        "Confezione della batteria al litio": None,
        "Peso articolo": None,
        "Aggiornamenti software garantiti fino a": None
    }
    try:
        tech_details_section = soup.find("div", attrs={'id': 'prodDetails'})
        if tech_details_section:
            rows = tech_details_section.find_all("tr")
            
            for row in rows:
                key = row.find("th")
                value = row.find("td")
                if key and value:
                    key_text = key.text.strip()
                    value_text = value.text.strip().replace("\u200e", "")
                    if key_text in details:
                        details[key_text] = value_text
            
    except AttributeError:
        pass

    return details

In [ ]:
if __name__ == '__main__':
    d = {
        "title": [],
        "price": [],
        "not discounted price": [],
        "rating": [],
        "reviews": [],
        "availability": [],
        "Marca": [],
        "Produttore": [],
        "serie": [],
        "colore": [],
        "Dimensioni del collo": [],
        "Pile": [],
        "Numero modello articolo": [],
        "Fattore di forma": [],
        "Dimensione schermo": [],
        "Risoluzione schermo": [],
        "Risoluzione": [],
        "Marchio processore": [],
        "Tipo processore": [],
        "Velocità processore": [],
        "Numero processori": [],
        "Dimensione RAM": [],
        "Tecnologia di memoria": [],
        "Tipologia di memoria computer": [],
        "Memoria massima supportata": [],
        "Dimensioni Hard-Disk": [],
        "Descrizione Hard-Disk": [],
        "Interfaccia Hard-Disk": [],
        "Dettagli audio": [],
        "Coprocessore grafico": [],
        "Grafica Chipset Brand": [],
        "Descrizione scheda grafica": [],
        "Tipo memoria scheda grafica": [],
        "Dimensioni memoria scheda grafica": [],
        "Interfaccia scheda grafica": [],
        "Tipo wireless": [],
        "Numero di porte USB 2.0": [],
        "Numero di porte USB 3.0": [],
        "Numero di porte HDMI": [],
        "Numero di porte Ethernet": [],
        "Voltaggio": [],
        "Wattaggio": [],
        "Tipologia di periferica ottica": [],
        "Posteriore Webcam Risoluzione": [],
        "Frontale Webcam Risoluzione": [],
        "Sistema operativo": [],
        "Le batterie sono incluse": [],
        "Capacità della batteria al litio": [],
        "La vita media della batteria (in ore)": [],
        "Confezione della batteria al litio": [],
        "Peso articolo": [],
        "Aggiornamenti software garantiti fino a": []
    }

    for page in range(1, 21):
        try:
            content = make_request(page)
            soup = BeautifulSoup(content, "html.parser")
            links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})
            links_list = [link.get('href') for link in links]

            for link in links_list:
                new_webpage = requests.get("https://www.amazon.it" + link, headers=headers)
                new_soup = BeautifulSoup(new_webpage.content, "html.parser")

                title = get_title(new_soup)
                price = get_price(new_soup)
                discount = get_not_discounted_price(new_soup)
                rating = get_rating(new_soup)
                review_count = get_review_count(new_soup)
                availability = get_availability(new_soup)
                technical_details = get_technical_details(new_soup)

                # Print each data to check if it's working
                print("Title:", title)
                print("Price:", price)
                print("Not Discounted Price:", discount)
                print("Rating:", rating)
                print("Number of Reviews:", review_count)
                print("Availability:", availability)
                print("Technical Details:", technical_details)
                print("-" * 50)

                # Append the main details to the dictionary
                d['title'].append(title)
                d['price'].append(price)
                d['not discounted price'].append(discount)
                d['rating'].append(rating)
                d['reviews'].append(review_count)
                d['availability'].append(availability)
                
                # Append the technical details to the dictionary
                for key, value in technical_details.items():
                    d[key].append(value)
            
            time.sleep(2)  # To prevent overwhelming the server
        except Exception as e:
            print(f"An error occurred on page {page}: {e}")
            continue

    # At this point, `d` contains all the extracted data
    print("Extraction complete!")

In [25]:
# Converti il dizionario in un DataFrame pandas
amazon_df = pd.DataFrame.from_dict(d)

# Nome del file CSV
csv_filename = "amazon_data.csv"

# Verifica se il file CSV esiste già
if os.path.isfile(csv_filename):
    # Se il file esiste, leggi i dati esistenti
    existing_df = pd.read_csv(csv_filename)
    # Aggiungi i nuovi dati al DataFrame esistente
    updated_df = pd.concat([existing_df, amazon_df], ignore_index=True)
else:
    # Se il file non esiste, utilizza semplicemente il DataFrame appena creato
    updated_df = amazon_df

# Salva il DataFrame aggiornato nel file CSV
updated_df.to_csv(csv_filename, index=False)